In [62]:
# Import needed packages
import os
import pandas as pd
import tweepy
import re
import string
from textblob import TextBlob
import preprocessor as p
import csv
import sys
import pandas_datareader as pdr
import datetime
from nltk.corpus import stopwords
import nltk

In [93]:
startdate = datetime.datetime(2022,3,30)
enddate = datetime.datetime(2022,4,5)
st = pdr.get_data_yahoo('MMM',start=startdate,end=enddate)
st

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2022-03-30,153.259995,150.770004,151.330002,151.429993,2395100,151.429993
2022-03-31,151.250000,148.800003,150.449997,148.880005,3193800,148.880005
2022-04-01,149.860001,146.850006,149.630005,149.690002,2196400,149.690002
2022-04-04,150.009995,146.080002,149.059998,149.529999,2631800,149.529999
2022-04-05,150.630005,148.509995,148.649994,148.899994,2148600,148.899994


In [26]:
# Get Twitter data and do sentiment analysis
# Authenticate credentials

#Twitter credentials for the app
consumer_key = 'aR8jQP7cfr623h9apTvM711pm'
consumer_secret = 'SLKF8TOUL8PtapZ5amz4Y9ZBRE99ocRxwvEnzIjfEFri2TbhTI'
access_key= '1511751059332493317-eFiU4zDgdWqZPDC7EWZrQVaCLJ5ZX8'
access_secret = 'U3yAxDcXSFVHqsbJW1adpZiElFoIBosY7FW1UJpAOULqM'

#Pass twitter credentials to tweepy
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_key,access_secret)
# api = tweepy.API(auth, wait_on_rate_limit = True, wait_on_rate_limit_notify=True)
api = tweepy.API(auth)

query = "$"+"MMM" + " -filter:retweets -filter:replies"
tweets = tweepy.Cursor(api.search_tweets ,q=query,lang='en', since = startdate, until = enddate, tweet_mode='extended').items(5)
print(tweets)
for tweet in tweets:
    status=tweet._json
    created_at = status['created_at']
    tweet_id = status['id_str']
    tweet_text = status['full_text']
    tweet_text = re.sub(r'\W', ' ',tweet_text)
    print(created_at)

Unexpected parameter: since


Thu Mar 31 22:51:05 +0000 2022
Thu Mar 31 22:30:02 +0000 2022
Thu Mar 31 20:28:37 +0000 2022
Thu Mar 31 17:33:06 +0000 2022
Tue Mar 29 23:03:05 +0000 2022


In [9]:
# Obtain stock tickers

data = pd.read_html('https://en.wikipedia.org/wiki/List_of_S&P_500_companies')
#print(data)
table = data[0]
#table.head()
tickers = table['Symbol'].tolist()
tickers1 = tickers[:49]
tickers2 = tickers[49:99]
tickers3 = tickers[99:149]
tickers4 = tickers[149:199]
tickers5 = tickers[199:249]
tickers6 = tickers[249:299]
tickers7 = tickers[349:399]
tickers8 = tickers[449:490]
tickers9 = tickers[480:500]
tickers10 = tickers[500:]


In [4]:
# Get stock prices and returns
startdate = datetime.datetime(2022,3,30)
enddate = datetime.datetime(2022,4,5)

file = open("stocks.csv",'w',newline='')
print(len(tickers))
count = 0
for ticker in tickers:
    count +=1
    print(count)
    try:
        st = pdr.get_data_yahoo(ticker,start=startdate,end=enddate)
        st['pct_change'] = st['Adj Close'].pct_change(4)
        #print(ticker + str(st['Adj Close']) + str(st['pct_change']))
        stockdata = csv.writer(file)
        stockdata.writerow([ticker,st['pct_change'].iloc[-1]])
    except:
        pass



505
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276


In [104]:
# Get Twitter data and do sentiment analysis
# Authenticate credentials

#Twitter credentials for the app
# consumer_key = 'ELRou1VKpS5p7EBEGamXqqGRT'
# consumer_secret = 'QbsToHJOyA9ZsDrFCCw1aErF9LubwqlfLpLcJviruhKepQwX3D'
# access_key= '3138259520-6jOClvoHXzF5DCf2XaylTxhrn1Rb7X50ykQyU3C'
# access_secret = 'r9bokM5UvRbS8QnsYdQksSi2tO8gRj7T4sfq2K41jzmQM'

# consumer_key = 'aR8jQP7cfr623h9apTvM711pm'
# consumer_secret = 'SLKF8TOUL8PtapZ5amz4Y9ZBRE99ocRxwvEnzIjfEFri2TbhTI'
# access_key= '1511751059332493317-eFiU4zDgdWqZPDC7EWZrQVaCLJ5ZX8'
# access_secret = 'U3yAxDcXSFVHqsbJW1adpZiElFoIBosY7FW1UJpAOULqM'

# consumer_key = 'Rd9nlZE2GWgkXXj6LfCGnWCtq'
# consumer_secret = 'yCQeUkiwCUtgsvdn46jQiK6yqISlXy8xlbV3PG4rr8sb3qgWpZ'
# access_key= '1432859916277063683-ZAXekHTSaIi3IecnfXAAlR1iCDWEO6'
# access_secret = 'LtxEkUYYuuRAePwCJaDhexTa2KwKn8nem26SZSHsfc2bu'

consumer_key = 'lLoOz2cbplDwX3vTzZFv2Hyqz'
consumer_secret = 'Yp5YFPvY3ISI2KUVQC6HAqYjDCUWw5na3IFrCvTVoTuvF9QddN'
access_key= '3028339548-i1TWGaqfItqlybCtkJZUQ7km95pylMZl8zPQBLb'
access_secret = 'llC8DaUitlD5wMl0FMqSxR3L78q0OaF56oazKJF5Hqnnr'





#Pass twitter credentials to tweepy
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_key,access_secret)
# api = tweepy.API(auth, wait_on_rate_limit = True, wait_on_rate_limit_notify=True)
api = tweepy.API(auth)

file = open("searchTweets233.csv","w",newline='')
alltweets = csv.writer(file)
count = 0
for ticker in tickers1:
    count += 1
    print(count)
    query = "$"+ticker + " -filter:retweets -filter:replies"
    tweets = tweepy.Cursor(api.search_tweets ,q=query,lang='en', until = enddate, tweet_mode='extended').items(20)
    for tweet in tweets:
        status=tweet._json
        created_at = status['created_at']
        tweet_id = status['id_str']
        tweet_text = status['full_text']
        tweet_text = re.sub(r'\W', ' ',tweet_text)
        print(tweet_text)
        try: 
            alltweets.writerow([ticker,created_at,tweet_id, tweet_text])
        except UnicodeEncodeError:
            pass




1
 MMM open interest for maturity 04 08 2022  High put 145 00 High call 152 50 PutCallRatio 0 91  maxpain  options https   t co aHXahUSCdG https   t co cmojteFgiv
   MMM price target update    Target Lowered by Barclays From  170 00 to  155 00  Day quote change   149 33   0 23   Target upside  3 8   Published  April 4  2022 https   t co IQv1jXSybp
 MMM 13 has crossed above    48 and ERROR retrieving 200 data  check manually  for the 15m intverval  12 00 02 PM
   MMM price target update    Target Lowered by Barclays From  170 00 to  155 00  Day quote change   148 4   0 84   Target upside  4 45   Published  April 4  2022 https   t co IQv1jXSybp
Stock   MMM        REALTIME TREND     BUY BUY   95 00   SELL   5 00   Try now the Best Pivot Point Calculator   https   t co A5q3ahyp0L  tradingview  stocks  stockstowatch
Stock   MMM        REALTIME TREND     BUY BUY   95 00   SELL   5 00   Try now the Best Pivot Point Calculator   https   t co Q5ES63osKo  tradingview  stocks  stockstowatch
Nice 

13
Insider Joseph L Moskowitz reports selling 300 shares of  AFL for a total cost of  19 461 00 https   t co fEbZAxysa7
Nice print for  AFL Size  341106 Price  65 5 Amount   22 342 443 00 Time  1830 See more  https   t co Tniza8nSU0 Join https   t co JWvlYbdLod to get REAL TIME prints  Runners    https   t co VC1LdPHaJR Losers    https   t co 9MWV4THfhQ Gappers    https   t co bHQKF1W40n
14
 A Max Pain 140 00  Maturity 04 14 2022   maxpain  options https   t co JWdNPrRAld https   t co y4g4Y6fCmJ
 a  15s  delayed   Issued Press Release on April 04  13 00 00  ACC 22  Medtronic renal denervation system demonstrates significant blood pressure reduction through three years https   t co ZxV3MjcuHv
 a  15s  delayed   Issued Press Release on April 04  12 57 00  DoubleVerify to Announce First Quarter 2022 Financial Results on May 10  2022 https   t co RzGEG8I8Dd
 a  15s  delayed   Issued Press Release on April 04  11 00 00  UMH PROPERTIES  INC  WILL HOST FIRST QUARTER 2022 FINANCIAL RESULTS WEB

Stock   GOOGL        REALTIME TREND     BUY BUY   95 00   SELL   5 00   Try now the Best Pivot Point Calculator   https   t co FweNBY9u20  tradingview  stocks  stockstowatch
 GOOGL  GOOG Based on 30 Wall Street analysts offering 12 month price targets for Alphabet Class A in the last 3 months  The average price target is  3 490 00 with a high forecast of  3 900 00
 GOOGL Evening Star Daily appearances since 2010  13 Close 5 days later    Avg   14 14 SD  69 29 Worst   202 04 Best  79 55  10 days later  Avg   30 00 SD  82 24 Worst   211 02 Best  67 37  30 days later  Avg   14 89 SD  128 87 Worst   399 96 Best  84 89
 KGKG the time is here  get in early with  KGKG to  1 00   gme  amc  sndl  amzn  spy  qqq  goog  googl  riot  Bitcoin get in with  KGKG https   t co jWimCHE61A
 GOOGL 13 has crossed below    48 and ERROR retrieving 200 data  check manually  for the 15m intverval  02 00 03 PM
26
 GOOGL  GOOG two pivots of interest are the 2850 49 and 2885 00 levels  Ideally the 10EMA bounce ho

 AMD 351 AMD Apr22 111 Calls  2 878  Above Ask    MULTI  10 09 00 135 IV 54 0   3 5 C2 77 x  2 83    2 87 x 42 BZX Volume 676  Premium  101k
Okay we got 100    which will do  MONDAY DAY TRADE WATCHLIST     AAPL calls  gt 175 73 puts  lt 173 33  AMD calls  gt 111 85 puts  lt 106 10  AMZN calls  gt 3327 40 puts  lt 3233 98  MSFT calls  gt 311 23 puts  lt 307 39  NVDA calls  gt 269 25 puts  lt 262 67  TSLA calls  gt 1117 00 puts  lt 1053 60  Good luck   https   t co IbQuMBwXPi
 AMD Option to look at for Monday         200 00 Call for Friday  January 20  2023  Roughly 627 Thousand dollars       Learn more on StockOrbit  Link on profile    
 AMD Option to look at for Monday   ðŸ   ðŸ     135 00 Call for Friday  May 20  2022  Roughly 389 Thousand dollars  ðŸ  ðŸ   Learn more on StockOrbit  Link on profile    
 AMD Option to look at for Monday         200 00 Call for Friday  January 20  2023  Roughly 627 Thousand dollars       Learn more on StockOrbit  Link on profile    
31
Insider Richard J

36
Nice print for  AWK Size  121584 Price  166 25 Amount   20 213 340 00 Time  1900 See more  https   t co rlC9JGAqzN Join https   t co JWvlYbejdL to get REAL TIME prints  Runners    https   t co VC1LdPHIzp Losers    https   t co 9MWV4THN7o Gappers    https   t co bHQKF1WBPV
 AWK  15s  delayed   Issued Press Release on April 04  13 30 00  American Water Recognized for Its Leadership and Commitment to Employees  amp amp  Communities https   t co tOIsckoBEs
 AWK  15s  delayed   Issued Press Release on April 04  10 13 00  Pennsylvania American Water to Plant Trees for Customers Who Switch to Paperless Billing in April https   t co BUoqbMTnKZ
 AWK  15s  delayed   Issued Press Release on March 31  11 30 00  American Water Launches New  Improved Digital Features to Benefit Customers https   t co vz4SOH8tvd
 AWK open interest for maturity 04 14 2022  High put 175 00 High call 170 00 PutCallRatio 1 58  maxpain  options https   t co WcbIW1BtkR https   t co m3NNqRXgim
Nice print for  AWK Size  1

            AAPL Put         Strike  200 00         Exp  05 20 22         Ask  22 95         Bid  22 65         OpenInterest  529         Volume  2 325   TOS    AAPL220520P200  https   t co Id1d5j5Ds2  RHD  https   t co TKI6MIM1PJ https   t co 93iEaY26jU
 AAPL 05 20 22  200 00 PUT  Type  large Time  14 00 41 Total Size  1150 Contracts Total Value   2635800 Average Price   22 92 Ask   23 25 Bid   22 85 Underlying   177 75 Sentiment  neutral
  Today s Top 10 Active  Stocks   Mon  Apr 04  2022   01 00 PM EST 1   TWTR   29 56   2   NIO   7 10   3   DIDI   3 72   4   AMD   1 20   5   AAPL   1 85   6   AMC   1 89   7   F   0 39   8   TLRY   0 40   9   BABA   6 36   10   TELL   4 73  
 AAPL Strike  115 00 Expiration  2024 01 19 Put  AAPL240119P115 Price  4 95 Size  1200 Time  2022 04 04 1 47 09PM Premium   594 000 00
 AAPL 01 19 24  120 00 CALL  Type  block Time  12 01 00 Total Size  186 Contracts Total Value   1248990 Average Price   67 15 Ask   67 8 Bid   67 15 Underlying   177 15 Sentiment

In [12]:
# Text Processing

# Open tweets CSV
file = open("searchTweets.csv","r")
tweets = pd.read_csv(file)

# Import stopwords with nltk.
stop = stopwords.words('english')

# Exclude stopwords with Python's list comprehension and pandas.DataFrame.apply.
tweets['tweet_without_stopwords'] = tweets['tweet'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

# Text Normalization
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()

def lemmatize_text(text):
    words = [lemmatizer.lemmatize(w) for w in w_tokenizer.tokenize(text)]
    return " ".join(words)
tweets['processed_tweet'] = tweets.tweet_without_stopwords.apply(lemmatize_text)


tweets[['processed_tweet']].values[0][0]


'MMM open interest maturity 04 08 2022 High put 145 00 High call 152 50 PutCallRatio 0 91 maxpain option http co aHXahUSCdG http co cmojteFgiv'

In [13]:
# Sentiment analysis

file = open("searchTweets.csv","r")
alltweets = csv.reader(file)
print(alltweets)
sentiment_file = open("sentimenttweets.csv","w",newline='')
sntTweets=csv.writer(sentiment_file)
count = 0
for row in alltweets:
    processed_tweet = tweets[['processed_tweet']].values[count][0]
    blob = TextBlob(row[3])
    blob2 = TextBlob(processed_tweet)
    if blob2.sentiment.polarity>0:
        sntTweets.writerow([row[0],row[1],row[2],blob.sentiment.polarity,"positive", blob.sentiment.subjectivity, blob2.sentiment.polarity, blob2.subjectivity])
    elif blob2.sentiment.polarity<0:
        sntTweets.writerow([row[0],row[1],row[2],blob.sentiment.polarity,"negative", blob.sentiment.subjectivity, blob2.sentiment.polarity, blob2.subjectivity])    
    elif blob2.sentiment.polarity==0:
        sntTweets.writerow([row[0],row[1],row[2],blob.sentiment.polarity,"neutral", blob.sentiment.subjectivity, blob2.sentiment.polarity, blob2.subjectivity])
    count += 1
        
file.close()
sentiment_file.close()